# Wrapping into Functions<br>and Calling by Value

* We can defne functions in python and C/C++ to modularize the code.

* Python

In [ ]:
def main():
    print(add(1, 2))
    print(add('1', '2'))

    
def add(a, b):
    return a + b


if "__main__" == __name__ :
    main()    

* C/C++

``` C++
#include <iostream>

using namespace std;

// Function prototypes
// Information to call the functions
// return type, function name, arguments, and argument types
int add_int(int a, int b);
double add_double(double a, double b);

int main(int argn, char *argv[]){
    cout << add_int(1, 2) << '\n';
    cout << add_double(1.0, 2.0) << '\n';
    
    return 0;
}


int add_int(int a, int b){
    return a + b;
}


double add_double(double a, double b){
    return a + b;
}

```



## Scope

* Variables within a function is valid only within that function.

* Python

In [ ]:
a = 0
c = 'zzz'


def main():

    print(add(1, 2))
    print(add('a', 'b'))
    print(f'main() : c = {c}')

    
def add(a, b):

    c = a + b
    print(f'add() : c = {c}')

    return c


if "__main__" == __name__ :
    main()



* C/C++

``` C++
#include <iostream>

using namespace std;

int add_int(int a, int b);
double add_double(double a, double b);


// Global variables
int a = 0;
char c[] = "zzz";


int main(int argn, char *argv[]){

    cout << "main() : c = " << c << '\n';
    cout << add_int(1, 2) << '\n';
    cout << add_double(1.1, 2.04) << '\n';
    cout << "main() : c = " << c << '\n';
    
    return 0;

}


int add_int(int a, int b){

    int c = a + b;

    cout << "add_int() : c = " << c << '\n';

    return c;

}


double add_double(double a, double b){

    double c = a + b;

    cout << "add_double() : c = " << c << '\n';

    return c;

}
```



### Resolving scope

* It is possible to access a global variable within a function even if the function has a local variable of the same name.

In [ ]:
a = 0
c = 'zzz'


def main():

    print(f'main() : c = {c}')
    print(add(1, 2))
    print(f'main() : c = {c}')
    print(add('a', 'b'))
    print(f'main() : c = {c}')

    
def add(a, b):
    global c

    c = a + b
    print(f'add() : c = {c}')

    return c


if "__main__" == __name__ :
    main()



``` C++
#include <iostream>

using namespace std;

int add_int(int a, int b);
double add_double(double a, double b);

int a = 0;
char c[] = "zzz";


int main(int argn, char *argv[]){
    
    cout << "main() : c = " << c << '\n';
    cout << add_int(1, 2) << '\n';
    cout << add_double(1.1, 2.04) << '\n';
    cout << "main() : c = " << c << '\n';
    
    return 0;
}


int add_int(int a, int b){
    int c = a + b;
    cout << "add_int() : c = " << c << '\n';
    cout << "add_int() : ::c = " << ::c << '\n';
    return c;
}


double add_double(double a, double b){
    double c = a + b;
    cout << "add_double() : c = " << c << '\n';
    return c;
}

```

## Exercise

### 00 Adding double precision floats

* Write python and C/C++ functions that can add two double precision floating point variables.
* In python and C/C++, write appropriate programs checking the result using `assert` or `assert()`.

### 01 Dot product of $1\times2$ vectors of double precision floats

* Write python and C/C++ functions that can calculate the dot product of two $1\times2$ double precision floating point vectors using `list`'s or arrays.
* In python and C/C++, write appropriate programs checking the result using `assert` or `assert()`.